In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle(f"dataset/3p6-selected.pkl", compression='zip')

In [3]:
df['xyz_id'] = df.groupby(['x', 'y', 'z']).ngroup()

In [4]:
df.head()

,x,y,z,vx,vy,vz,px,py,pz,time,distance,xyz_id
1824696,-117.0,87.0,-33.0,0.48,0.02,0.04,-0.57,-0.11,-2.17,301,150.0,2356
1824697,-113.0,87.0,-33.0,0.48,0.02,0.04,-0.50,-0.40,-2.10,301,146.0,3148
1824698,-109.0,87.0,-33.0,0.48,0.02,0.04,-0.45,-0.58,-2.01,301,143.0,3940
1824699,-105.0,87.0,-33.0,0.48,0.02,0.04,-0.42,-0.67,-1.91,301,140.0,4732
1824700,-101.0,87.0,-33.0,0.48,0.02,0.05,-0.36,-0.69,-1.80,301,137.0,5524


In [5]:
df.shape

(304116, 12)

In [6]:
df.time.unique()

array([301, 305, 300, 302, 303, 304])

In [7]:
train_df = df[df["time"] != 303]
train_df = train_df.drop(columns=["x","y","z","px","py","pz"], axis=1)

In [8]:
test_df = df[df["time"] == 303]
test_df = test_df.drop(columns=["x","y","z","px","py","pz"], axis=1)

In [9]:
train_df.shape, test_df.shape

((253430, 6), (50686, 6))

In [10]:
df[['x', 'y', 'z']].drop_duplicates().count()[0]

50686

In [38]:
centroid = np.array([0, 0, 0])

# calculate the Euclidean distance between each point and the centroid
df['distance_from_centroid'] = np.sqrt(
    (df['x'] - centroid[0])**2 +
    (df['y'] - centroid[1])**2 +
    (df['z'] - centroid[2])**2
)


In [39]:
df['distance'].nunique(), df['distance_from_centroid'].nunique()

(155, 10387)

In [14]:
train_df.head()

,vx,vy,vz,time,distance,xyz_id
1824696,0.48,0.02,0.04,301,150.0,2356
1824697,0.48,0.02,0.04,301,146.0,3148
1824698,0.48,0.02,0.04,301,143.0,3940
1824699,0.48,0.02,0.04,301,140.0,4732
1824700,0.48,0.02,0.05,301,137.0,5524


In [ ]:
# set constraint for time

In [11]:
import sdv

In [12]:
print(sdv.__version__)

1.0.0


In [37]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()

In [38]:
metadata.detect_from_dataframe(data=train_df)

In [39]:
python_dict = metadata.to_dict()
python_dict

{'columns': {'vx': {'sdtype': 'numerical'},
  'vy': {'sdtype': 'numerical'},
  'vz': {'sdtype': 'numerical'},
  'time': {'sdtype': 'numerical'},
  'distance': {'sdtype': 'numerical'},
  'xyz_id': {'sdtype': 'numerical'}},
 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}

In [40]:
# metadata.update_column(
#     column_name='xyz_id',
#     sdtype='id')
# metadata.set_primary_key(column_name='xyz_id')

In [41]:
metadata.validate()

In [42]:
from sdv.single_table import CopulaGANSynthesizer

In [ ]:
synthesizer = CopulaGANSynthesizer(metadata)
synthesizer.fit(train_df)

In [34]:
synthetic_data = synthesizer.sample(num_rows=10)

In [35]:
synthetic_data

,vx,vy,vz,time,distance,xyz_id
0,0.44,0.05,0.02,301,51.0,0
1,0.37,-0.07,0.08,302,95.0,1
2,0.48,0.06,-0.02,302,114.0,2
3,0.46,0.08,0.01,302,102.0,3
4,0.44,0.02,0.04,304,48.0,4
5,0.50,0.00,0.04,302,71.0,5
6,0.47,-0.01,0.03,304,79.0,6
7,0.28,0.02,-0.00,301,103.0,7
8,0.48,-0.11,-0.02,302,42.0,8
9,0.47,-0.01,0.05,304,123.0,9


In [36]:
train_df[(train_df["time"]==301) & (train_df["distance"]==51)]

,vx,vy,vz,time,distance,xyz_id
1825493,0.45,0.04,-0.01,301,51.0,25108
1825494,0.45,0.05,-0.01,301,51.0,25900
1825616,0.50,-0.04,0.04,301,51.0,21112
1825805,0.60,-0.10,0.00,301,51.0,18682
1825822,0.49,0.05,0.08,301,51.0,32146
...,...,...,...,...,...,...
1874397,0.26,-0.04,0.07,301,51.0,31173
1874447,0.36,0.00,-0.02,301,51.0,20067
1874460,0.29,-0.01,0.03,301,51.0,30363
1874579,0.41,-0.03,-0.06,301,51.0,23199
